In [1]:
import pandas as pd
import numpy as np
from datetime import datetime

from modules.info import *  # 导入时间、路径信息
from modules.save2csv import *  # 导入保存函数


In [2]:
# 读取需要的数据字典
keys_needed = pd.read_excel(rule_path + '数据字典_230407.xlsx')

In [3]:
# 读取并处理底层数据

entity_3 = pd.DataFrame()

# 共需要处理三年的数据
years = ['2021', '2020', '2019']

for year in years:
    # 读取底层数据
    fin_data = pd.read_csv(data_path + 'fin_data_' + year + '.csv')
    fin_data_wind = pd.read_excel(data_path + 'fin_data_wind_' + year + '.xlsx')
    fin_entity = pd.read_excel(data_path + 'fin_entity.xlsx')
    fin_master = pd.read_excel(data_path + 'fin_master_' + year + '.xlsx')

    # fin_data
    fin_data_wide = fin_data.pivot(index='master_id', columns='code', values='end_value')
    fin_data_wide.reset_index(inplace=True)
    fin_data_wide.rename(columns={'master_id': 'id'}, inplace=True)
    
    # fin_data_wind
    fin_data_wind_wide = fin_data_wind.pivot(index='master_id', columns='code', values='end_value')
    fin_data_wind_wide.reset_index(inplace=True)
    fin_data_wind_wide.rename(columns={'master_id': 'id'}, inplace=True)
    
    # fin_entity
    fin_entity = fin_entity[['id', 'entity_name', 'stock_code', 'stock_name', 'uniform_credit', 'industry_whitewash']]
    fin_entity.rename(columns={'id': 'entity_id'}, inplace=True)
    
    # fin_master
    fin_master = fin_master[['id', 'entity_id']]
    
    # 合并当年主体数据宽表
    entity = pd.merge(fin_entity, fin_master, on='entity_id')
    entity = pd.merge(entity, fin_data_wide, on='id')
    entity = pd.merge(entity, fin_data_wind_wide, on='id')
    entity['year'] = int(year)
    
    # 合并入主体总表
    entity_3 = pd.concat([entity_3, entity])

/var/folders/21/r5tywv_91vz_8s1tgrs4qpjm0000gn/T/ipykernel_12964/351913907.py:10: DtypeWarning: Columns (2,5) have mixed types. Specify dtype option on import or set low_memory=False.
  fin_data = pd.read_csv(data_path + 'fin_data_' + year + '.csv')


In [4]:
# 调整年份列位置
last_column = entity_3.pop(entity_3.columns[-1])
entity_3.insert(1, last_column.name, last_column)

# 删除多余科目
drop_cols = entity_3.columns[7:][~entity_3.columns[7:].isin(keys_needed.iloc[:, 0])]
entity_3.drop(drop_cols, axis=1, inplace=True)
# entity_3.drop('id', axis=1, inplace=True)

# 打印出缺少的科目
missing_cols = keys_needed.iloc[:, 0][~keys_needed.iloc[:, 0].isin(entity_3.columns)]
print(missing_cols.tolist())

# 缺少的科目补充并默认赋值为0
entity_3[missing_cols] = 0

# 补充缺失的审计意见 TODO: 数据缺失
entity_3['NOTES_AOO_AuditOpinion_idou'] = np.nan
entity_3['EN_ANNUAL_MarketCapitalization_idou'] = np.nan


entity_3.head(2)

['BS_0213_Aquaculture', 'BS_0213_LandUse', 'BS_0213_Mining']


,entity_id,year,entity_name,stock_code,stock_name,uniform_credit,industry_whitewash,BS_0104,BS_0105,BS_0108,...,stmnote_Eoitems_24,stmnote_Rdexp,stmnote_assetdetail_2,stmnote_incometax_6,stmnote_profitapr_3,BS_0213_Aquaculture,BS_0213_LandUse,BS_0213_Mining,NOTES_AOO_AuditOpinion_idou,EN_ANNUAL_MarketCapitalization_idou
0,19718,2021,万科企业股份有限公司,000002.SZ,万科A,91440300192181490G,M009,4743597023.95,67230335116.78,0,...,142251501.17,642366251.89,5071843044.58,14153104184.93,0,0,0,0,NaN,NaN
1,19719,2021,深圳国华网安科技股份有限公司,000004.SZ,国农科技,91440300192441969E,M017,370602392.09,35673070.1,11316734.8,...,12096041.51,47845162.91,4827463.26,6379802.84,0,0,0,0,NaN,NaN


In [5]:
save2csv(entity_3, '0_处理好的主体')